In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130610

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,185949.4744
2018-02-28,184035.0149
2018-03-31,153547.6396
2018-04-30,109978.6400
2018-05-31,104834.4066
2018-06-30,121046.4956
2018-07-31,111731.1606
2018-08-31,122624.1074
2018-09-30,124158.9274


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    185949.4744
 2018-02-28    184035.0149
 2018-03-31    153547.6396
 2018-04-30    109978.6400
 2018-05-31    104834.4066
 2018-06-30    121046.4956
 2018-07-31    111731.1606
 2018-08-31    122624.1074
 2018-09-30    124158.9274
 2018-10-31    122963.4419
 2018-11-30    204651.7396
 2018-12-31    135666.6754
 2019-01-31    256900.6388
 2019-02-28    173348.1697
 2019-03-31    226191.7071
 2019-04-30    203889.9855
 2019-05-31    153699.7246
 2019-06-30    162650.2492
 2019-07-31    150613.7263
 2019-08-31    197885.2094
 2019-09-30    132891.1060
 2019-10-31    167309.8288
 2019-11-30    193524.4987
 2019-12-31    174283.7862
 2020-01-31    194606.5907
 2020-02-29    176063.3015
 2020-03-31    224862.3686
 2020-04-30    193962.6206
 2020-05-31    202345.4879
 2020-06-30    256591.9027
 2020-07-31    275751.0112
 2020-08-31    297525.1328
 2020-09-30    314746.4940
 2020-10-31    347349.9102
 2020-11-30    337906.5299
 2020-12-31    313853.8509

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.477761
p-value : 0.544482
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.279761
p-value : 0.015808
Critical Values :
	1%: -3.632743
	5%: -2.948510
	10%: -2.613017


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=996.933, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=994.088, Time=0.13 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=995.567, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=996.178, Time=0.07 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=998.798, Time=0.10 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.55 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=993.294, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=995.094, Time=0.08 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=997.085, Time=0.09 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=1001.705, Time=0.13 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=995.089, Time=0.06 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=997.176, Time=0.13 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=991.807, Time=0.22 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.37 sec
 ARIMA(3,1,0)(0,0,0)[1] intercept   

ARIMA(order=(2, 1, 2), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,1,2))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(2, 1, 2)   Log Likelihood                -491.413
Method:                       css-mle   S.D. of innovations          36751.909
Date:                Sat, 11 Sep 2021   AIC                            994.825
Time:                        04:47:07   BIC                           1005.107
Sample:                             1   HQIC                           998.569
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       -127.3943   7012.490     -0.018      0.986   -1.39e+04    1.36e+04
ar.L1.D.y      0.5893      0.162      3.632      0.000       0.271       0.907
ar.L2.D.y     -0.4153      0.164     -2.528      0.0

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([211468.254785  , 227339.91534283]), array([36751.90945294, 43109.9727958 ]), array([[139435.83589416, 283500.67367585],
       [142845.92128857, 311833.90939709]]))
